In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step0. Loading library 

In [1]:
!pip install gensim

In [2]:
!pip install konlpy

     ---------------------------------------- 19.4/19.4 MB 8.0 MB/s eta 0:00:00
     ---------------------------------------- 3.6/3.6 MB 8.5 MB/s eta 0:00:00
     -------------------------------------- 362.6/362.6 KB 7.5 MB/s eta 0:00:00


In [9]:
!pip install wordcloud

  Using cached wordcloud-1.8.1.tar.gz (220 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for wordcloud
Failed to build wordcloud
  Running setup.py install for wordcloud: started
  Running setup.py install for wordcloud: finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-3.9
      creating build\lib.win-amd64-3.9\wordcloud
      copying wordcloud\color_from_image.py -> build\lib.win-amd64-3.9\wordcloud
      copying wordcloud\tokenization.py -> build\lib.win-amd64-3.9\wordcloud
      copying wordcloud\wordcloud.py -> build\lib.win-amd64-3.9\wordcloud
      copying wordcloud\wordcloud_cli.py -> build\lib.win-amd64-3.9\wordcloud
      copying wordcloud\_version.py -> build\lib.win-amd64-3.9\wordcloud
      copying wordcloud\__init__.py -> build\lib.win-amd64-3.9\wordcloud
      copying wordcloud\__main__.py -> build\lib.win-amd64-3.9\wordcloud
      copying wordcloud\stopwords -> build\lib.win-amd64-3.9\wordcloud
      copying wordcloud\DroidSansMono.ttf -> build\lib.win-amd64-3.9\wordcloud


In [3]:
!pip install pyLDAvis

In [11]:
import pandas as pd
import datetime
import time
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter
from konlpy.tag import Kkma
from wordcloud import WordCloud
%matplotlib inline
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
import pyLDAvis
import pyLDAvis.gensim_models
from gensim import corpora, models
import gensim
from gensim.models import CoherenceModel
from gensim import corpora, models
from collections import Counter #데이터의 개수를 정리할 수 있는 모듈


C:\Users\ra4\anaconda3\envs\dilab\lib\site-packages\gensim\matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


In [12]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [13]:
pd.set_option('mode.chained_assignment',  None)

# Step1. Loading Data 

In [14]:
nbs = pd.read_excel('crawling_data_all.xlsx')
nbs.head()

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Document Type,Publication Stage,Open Access,Source,EID,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,"Jouin C., Mlocek Z.",57219668257;57219668269;,Evolutions of nature based solutions for storm...,2020,Techniques - Sciences - Methodes,115,3,NaN,61.0,70.0,...,Article,Final,NaN,Scopus,2-s2.0-85094632803,NaN,NaN,NaN,NaN,NaN
1,Buzási A.,56692966000;,Climate vulnerability and adaptation challenge...,2021,Climate,9,2,25,1.0,17.0,...,Article,Final,NaN,Scopus,2-s2.0-85100598191,NaN,NaN,NaN,NaN,NaN
2,"Yu G.-R., Yang M., Chen Z., Zhang L.-M.",55807480400;55703268000;57001631400;55077923500;,Technical approach and strategic plan for larg...,2021,Chinese Journal of Applied Ecology,32,4,NaN,1141.0,1153.0,...,Article,Final,NaN,Scopus,2-s2.0-85104538541,NaN,NaN,NaN,NaN,NaN
3,"Thorn J.P.R., Biancardi Aleu R., Wijesinghe A....",56613034600;57260808600;57226747891;5726048210...,Mainstreaming nature-based solutions for clima...,2021,Landscape and Urban Planning,216,NaN,104235,NaN,NaN,...,Article,Final,NaN,Scopus,2-s2.0-85115004828,NaN,NaN,NaN,NaN,NaN
4,"Schuerch M., Spencer T., Temmerman S., Kirwan ...",54783506900;55156903100;6602902860;10044268300...,Future response of global coastal wetlands to ...,2018,Nature,561,7722,NaN,231.0,234.0,...,Article,Final,NaN,Scopus,2-s2.0-85053248555,NaN,NaN,NaN,NaN,NaN


In [92]:
df = nbs[['Title','Year','Abstract','']]  # 
df.head()

,Title,Year,Abstract
0,Evolutions of nature based solutions for storm...,2020,Stormwater management is one of the greatest c...
1,Climate vulnerability and adaptation challenge...,2021,Wine producers face several challenges regardi...
2,Technical approach and strategic plan for larg...,2021,Terrestrial ecosystems are the core components...
3,Mainstreaming nature-based solutions for clima...,2021,(Despite a growing recognition of the importan...
4,Future response of global coastal wetlands to ...,2018,The response of coastal wetlands to sea-level ...


In [93]:
uniqueyears, time_slices = np.unique(df.Year, return_counts=True)
print(uniqueyears, time_slices)

[2003 2006 2007 2009 2010 2012 2014 2015 2016 2017 2018 2019 2020 2021
 2022] [  1   1   1   1   1   2   5   7  20  51  73 123 263 485  22]


In [94]:
# 2016년 이후 논문만 추출 
df = df[df['Year'] >= 2016]
uniqueyears, time_slices = np.unique(df.Year, return_counts=True)
print(uniqueyears, time_slices)

[2016 2017 2018 2019 2020 2021 2022] [ 20  51  73 123 263 485  22]


In [116]:
sum(time_slices)

1037

In [95]:
df = df.reset_index(drop = True)
df.head()

,Title,Year,Abstract
0,Evolutions of nature based solutions for storm...,2020,Stormwater management is one of the greatest c...
1,Climate vulnerability and adaptation challenge...,2021,Wine producers face several challenges regardi...
2,Technical approach and strategic plan for larg...,2021,Terrestrial ecosystems are the core components...
3,Mainstreaming nature-based solutions for clima...,2021,(Despite a growing recognition of the importan...
4,Future response of global coastal wetlands to ...,2018,The response of coastal wetlands to sea-level ...


# Step2. Data Cleaning
- 한글, 숫자, 특수문자 모두 제거  
- 고유명사, 문장 시작을 대문자인 영어의 특성을 고려해 모두 소문자로 변경  

In [96]:
# # 영문자 이외 문자(한글, 숫자, 특수문자)는 공백으로 변환
df['Abstract'] = df['Abstract'].str.replace('[^a-zA-Z]',' ',regex=True)

# 고유명사, 첫시작 대문자를 고려하여 모두 소문자로 변경 
df['Abstract'] = df['Abstract'].str.lower()
df.head()

,Title,Year,Abstract
0,Evolutions of nature based solutions for storm...,2020,stormwater management is one of the greatest c...
1,Climate vulnerability and adaptation challenge...,2021,wine producers face several challenges regardi...
2,Technical approach and strategic plan for larg...,2021,terrestrial ecosystems are the core components...
3,Mainstreaming nature-based solutions for clima...,2021,despite a growing recognition of the importan...
4,Future response of global coastal wetlands to ...,2018,the response of coastal wetlands to sea level ...


# Step3. Word tokenization & Extract noun & Lemmatization
- 토큰화 
- 각 토큰마다 품사 태깅 후, 명사만 설정해서 추출  
- 원형 찾아서 통일 => ex. change,changes => change 
- 한글자 제거  

In [ ]:
-import nltk
import pickle
from nltk.corpus import stopwords
import re
nltk.download('all')

In [97]:
# 1분 소요 

df['Abstract_token'] = None
df['Abstract_NN'] = None

for i in range(len(df)):
    df['Abstract_token'][i] = nltk.word_tokenize(df['Abstract'][i])   # Word Tokenization
    df['Abstract_token'][i] = nltk.pos_tag(df['Abstract_token'][i])   # POS tagging
    NN_words = []

    for word, pos in df['Abstract_token'][i]:
        if 'NN' in pos:
            NN_words.append(word)
        df['Abstract_NN'][i] = NN_words   # Extract noun

df.head()

,Title,Year,Abstract,Abstract_token,Abstract_NN
0,Evolutions of nature based solutions for storm...,2020,stormwater management is one of the greatest c...,"[(stormwater, NN), (management, NN), (is, VBZ)...","[stormwater, management, challenges, environme..."
1,Climate vulnerability and adaptation challenge...,2021,wine producers face several challenges regardi...,"[(wine, NN), (producers, NNS), (face, VBP), (s...","[wine, producers, challenges, climate, change,..."
2,Technical approach and strategic plan for larg...,2021,terrestrial ecosystems are the core components...,"[(terrestrial, JJ), (ecosystems, NNS), (are, V...","[ecosystems, core, components, life, productio..."
3,Mainstreaming nature-based solutions for clima...,2021,despite a growing recognition of the importan...,"[(despite, IN), (a, DT), (growing, VBG), (reco...","[recognition, importance, infrastructure, ecos..."
4,Future response of global coastal wetlands to ...,2018,the response of coastal wetlands to sea level ...,"[(the, DT), (response, NN), (of, IN), (coastal...","[response, wetlands, rise, twenty, century, sc..."


In [98]:
# Lemmaization(원형찾기) => 원형을 찾아 같은 의미의 단어 토큰들을 하나의 값으로 인지하도록 한다. 
wlem = nltk.WordNetLemmatizer()

for i in range(len(df)):

    lemmatized_words = []

    for word in df['Abstract_NN'][i]:
        new_word = wlem.lemmatize(word)
        lemmatized_words.append(new_word)
        df['Abstract_NN'][i] = lemmatized_words
        
df.head()

,Title,Year,Abstract,Abstract_token,Abstract_NN
0,Evolutions of nature based solutions for storm...,2020,stormwater management is one of the greatest c...,"[(stormwater, NN), (management, NN), (is, VBZ)...","[stormwater, management, challenge, environmen..."
1,Climate vulnerability and adaptation challenge...,2021,wine producers face several challenges regardi...,"[(wine, NN), (producers, NNS), (face, VBP), (s...","[wine, producer, challenge, climate, change, i..."
2,Technical approach and strategic plan for larg...,2021,terrestrial ecosystems are the core components...,"[(terrestrial, JJ), (ecosystems, NNS), (are, V...","[ecosystem, core, component, life, production,..."
3,Mainstreaming nature-based solutions for clima...,2021,despite a growing recognition of the importan...,"[(despite, IN), (a, DT), (growing, VBG), (reco...","[recognition, importance, infrastructure, ecos..."
4,Future response of global coastal wetlands to ...,2018,the response of coastal wetlands to sea level ...,"[(the, DT), (response, NN), (of, IN), (coastal...","[response, wetland, rise, twenty, century, sca..."


In [99]:
# 1글자 제거 
df['Abstract_NN'] = df['Abstract_NN'].apply(lambda x: [word for word in x if (len(word) > 1 )])

df.head()

,Title,Year,Abstract,Abstract_token,Abstract_NN
0,Evolutions of nature based solutions for storm...,2020,stormwater management is one of the greatest c...,"[(stormwater, NN), (management, NN), (is, VBZ)...","[stormwater, management, challenge, environmen..."
1,Climate vulnerability and adaptation challenge...,2021,wine producers face several challenges regardi...,"[(wine, NN), (producers, NNS), (face, VBP), (s...","[wine, producer, challenge, climate, change, i..."
2,Technical approach and strategic plan for larg...,2021,terrestrial ecosystems are the core components...,"[(terrestrial, JJ), (ecosystems, NNS), (are, V...","[ecosystem, core, component, life, production,..."
3,Mainstreaming nature-based solutions for clima...,2021,despite a growing recognition of the importan...,"[(despite, IN), (a, DT), (growing, VBG), (reco...","[recognition, importance, infrastructure, ecos..."
4,Future response of global coastal wetlands to ...,2018,the response of coastal wetlands to sea level ...,"[(the, DT), (response, NN), (of, IN), (coastal...","[response, wetland, rise, twenty, century, sca..."


# Step4. Stopwords processing

In [100]:
#nltk에서 제공하는 불용어사전 이용
stopwords_list = stopwords.words('english')   # ex. I my me mine all should a ... 

In [ ]:
# stopwords_list

In [101]:
# 사용자 정의 불용어 사전 
customized_stopwords = ['nature', 'based', 'solutions', 'nbs', 'studing', 'studies', 'study','management', 'manage', 'model', 'use', 
                        'plan', 'increase','service',  'result', 'system', 'implement','implementation', 'treat', 'treatment', 'change','area']  # change, area 추가 

In [102]:
customized_stopwords_2 = ['city','approach','author','analysis','benefit','impact','effect','research','adaptation','planning','project']

In [103]:
stopwords_list.extend(customized_stopwords)

In [104]:
stopwords_list.extend(customized_stopwords_2)

In [105]:
 for i in range(len(df)):
     unique_NN_words = set(df['Abstract_NN'][i])
     final_NN_words = df['Abstract_NN'][i]

     for word in unique_NN_words:
         if word in stopwords_list:
             while word in final_NN_words: final_NN_words.remove(word)   # 불용어 제거 
     df['Abstract_NN'][i] = final_NN_words

In [106]:
df.head()

,Title,Year,Abstract,Abstract_token,Abstract_NN
0,Evolutions of nature based solutions for storm...,2020,stormwater management is one of the greatest c...,"[(stormwater, NN), (management, NN), (is, VBZ)...","[stormwater, challenge, environment, way, envi..."
1,Climate vulnerability and adaptation challenge...,2021,wine producers face several challenges regardi...,"[(wine, NN), (producers, NNS), (face, VBP), (s...","[wine, producer, challenge, climate, industry,..."
2,Technical approach and strategic plan for larg...,2021,terrestrial ecosystems are the core components...,"[(terrestrial, JJ), (ecosystems, NNS), (are, V...","[ecosystem, core, component, life, production,..."
3,Mainstreaming nature-based solutions for clima...,2021,despite a growing recognition of the importan...,"[(despite, IN), (a, DT), (growing, VBG), (reco...","[recognition, importance, infrastructure, ecos..."
4,Future response of global coastal wetlands to ...,2018,the response of coastal wetlands to sea level ...,"[(the, DT), (response, NN), (of, IN), (coastal...","[response, wetland, rise, twenty, century, sca..."


# Step5. TF-IDF 
- 단순히 단어의 빈도만 고려한다면 모든 문서에서 자주 쓰일 수 밖에 없는 단어들이 중요하다고 인식 될 수 있습니다. 
- 모든 초록에 등장하는 단어는 의미가 없으니 정제해주는 과정이 필요함  
- TF-IDF :  단어의 빈도(Term Frequency)와 역 문서 빈도(Inverse Document Frequency)를 토대로, 특정 문서 내에 어떤 단어가 얼마나 중요한 지를 나타내는 통계적 수치.

In [107]:
from sklearn.feature_extraction.text import CountVectorizer

In [108]:
df.head()

,Title,Year,Abstract,Abstract_token,Abstract_NN
0,Evolutions of nature based solutions for storm...,2020,stormwater management is one of the greatest c...,"[(stormwater, NN), (management, NN), (is, VBZ)...","[stormwater, challenge, environment, way, envi..."
1,Climate vulnerability and adaptation challenge...,2021,wine producers face several challenges regardi...,"[(wine, NN), (producers, NNS), (face, VBP), (s...","[wine, producer, challenge, climate, industry,..."
2,Technical approach and strategic plan for larg...,2021,terrestrial ecosystems are the core components...,"[(terrestrial, JJ), (ecosystems, NNS), (are, V...","[ecosystem, core, component, life, production,..."
3,Mainstreaming nature-based solutions for clima...,2021,despite a growing recognition of the importan...,"[(despite, IN), (a, DT), (growing, VBG), (reco...","[recognition, importance, infrastructure, ecos..."
4,Future response of global coastal wetlands to ...,2018,the response of coastal wetlands to sea level ...,"[(the, DT), (response, NN), (of, IN), (coastal...","[response, wetland, rise, twenty, century, sca..."


In [112]:
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

In [114]:
df['Abstract_str'] = None

for i in range(len(df)):
    df['Abstract_str'][i] = listToString(df['Abstract_NN'][i])

In [115]:
df.head()

,Title,Year,Abstract,Abstract_token,Abstract_NN,Abstract_str
0,Evolutions of nature based solutions for storm...,2020,stormwater management is one of the greatest c...,"[(stormwater, NN), (management, NN), (is, VBZ)...","[stormwater, challenge, environment, way, envi...",stormwater challenge environment way environme...
1,Climate vulnerability and adaptation challenge...,2021,wine producers face several challenges regardi...,"[(wine, NN), (producers, NNS), (face, VBP), (s...","[wine, producer, challenge, climate, industry,...",wine producer challenge climate industry vulne...
2,Technical approach and strategic plan for larg...,2021,terrestrial ecosystems are the core components...,"[(terrestrial, JJ), (ecosystems, NNS), (are, V...","[ecosystem, core, component, life, production,...",ecosystem core component life production activ...
3,Mainstreaming nature-based solutions for clima...,2021,despite a growing recognition of the importan...,"[(despite, IN), (a, DT), (growing, VBG), (reco...","[recognition, importance, infrastructure, ecos...",recognition importance infrastructure ecosyste...
4,Future response of global coastal wetlands to ...,2018,the response of coastal wetlands to sea level ...,"[(the, DT), (response, NN), (of, IN), (coastal...","[response, wetland, rise, twenty, century, sca...",response wetland rise twenty century scale pro...


In [132]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words="english",
                            use_idf=True,
                            ngram_range=(1,1), # 오직 1-grams만 고려함
                            min_df = 0.01,     #5% 미만으로 문서에서 출현하는 단어들은 삭제
                            max_df = 0.95)      #전체 문서의 70% 문서에 출현하는 단어들은 너무 빈번한 단어라고 판단하여 삭제  
                              

In [133]:
tfidf = vectorizer.fit_transform(df['Abstract_str'])

In [134]:
tf_idf_list = vectorizer.get_feature_names()
tf_idf_list

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['ability',
 'absence',
 'abundance',
 'academy',
 'acceptance',
 'access',
 'accessibility',
 'account',
 'accounting',
 'accumulation',
 'achievement',
 'action',
 'activity',
 'actor',
 'addition',
 'adoption',
 'advantage',
 'afforestation',
 'africa',
 'age',
 'agency',
 'agenda',
 'agent',
 'agreement',
 'agriculture',
 'aim',
 'air',
 'alternative',
 'america',
 'applicability',
 'application',
 'arrangement',
 'art',
 'article',
 'asia',
 'aspect',
 'assessment',
 'asset',
 'association',
 'assumption',
 'attention',
 'attenuation',
 'attitude',
 'attribute',
 'australia',
 'authority',
 'availability',
 'average',
 'awareness',
 'background',
 'balance',
 'bank',
 'barrier',
 'base',
 'basel',
 'baseline',
 'basin',
 'basis',
 'beach',
 'behavior',
 'beneficiary',
 'biodiversity',
 'biomass',
 'blue',
 'bod',
 'body',
 'boundary',
 'brazil',
 'breakwater',
 'budget',
 'buffer',
 'building',
 'business',
 'campaign',
 'canopy',
 'capability',
 'capacity',
 'capital',
 'carbon',

In [138]:
df['tfidf_NN'] = None 

for i in range(len(df)):
    result = pd.DataFrame(df['Abstract_NN'][i])
    result = result[result[0].str.contains('|'.join(tf_idf_list))][0].tolist()   # tf_idf_list에 해당되는 단어만 남겨두기 
    df['tfidf_NN'][i] = result

In [139]:
df['제거개수'] = None

for i in range(len(df)):
    df['제거개수'][i] = len(df['Abstract_NN'][i]) - len(df['tfidf_NN'][i])

In [140]:
df.head()

,Title,Year,Abstract,Abstract_token,Abstract_NN,Abstract_str,tfidf_NN,제거개수
0,Evolutions of nature based solutions for storm...,2020,stormwater management is one of the greatest c...,"[(stormwater, NN), (management, NN), (is, VBZ)...","[stormwater, challenge, environment, way, envi...",stormwater challenge environment way environme...,"[stormwater, challenge, environment, way, envi...",10
1,Climate vulnerability and adaptation challenge...,2021,wine producers face several challenges regardi...,"[(wine, NN), (producers, NNS), (face, VBP), (s...","[wine, producer, challenge, climate, industry,...",wine producer challenge climate industry vulne...,"[challenge, climate, industry, vulnerability, ...",14
2,Technical approach and strategic plan for larg...,2021,terrestrial ecosystems are the core components...,"[(terrestrial, JJ), (ecosystems, NNS), (are, V...","[ecosystem, core, component, life, production,...",ecosystem core component life production activ...,"[ecosystem, core, component, life, production,...",7
3,Mainstreaming nature-based solutions for clima...,2021,despite a growing recognition of the importan...,"[(despite, IN), (a, DT), (growing, VBG), (reco...","[recognition, importance, infrastructure, ecos...",recognition importance infrastructure ecosyste...,"[recognition, importance, infrastructure, ecos...",12
4,Future response of global coastal wetlands to ...,2018,the response of coastal wetlands to sea level ...,"[(the, DT), (response, NN), (of, IN), (coastal...","[response, wetland, rise, twenty, century, sca...",response wetland rise twenty century scale pro...,"[response, wetland, rise, century, scale, sea,...",17


In [141]:
noun_list = df['tfidf_NN'].tolist()

# Step5. LDA Modeling

## LDA Parameter 
1. topic : 토픽의 개수  
2. chunksize : 훈련 알고리즘에 사용할 문서 개수  => 빠른 학습이 필요하면, 청크 사이즈를 키우기, Hoffman 논문에 의하면 청크사이즈는 모델 품질에 영향을 미치치만 차이가 그렇게 크지 않다고 한다.  
3. passes : 모델 학습시 전체 코퍼스에서 모델을 학습시키는 빈도 제어 
4. iteration : 각 문서에 대해서 루푸를 얼마나 돌릴지  
5. pass & iteration은 클수록 좋은 편  
6. alpha, eta = auto  => 디리클레 분포의 감마함수에 대한 파라미터  

---
## LDA Measure  
1. Perplexity; 혼란도  
: 특정 확률 모델이 실제로 관측되는 값을 얼마나 잘 예측하는지를 뜻한다. 이 값이 작으면 토픽모델이 문서를 잘 반영된다고 알 수 있다.  **작은게 좋은 거**  

2. Coherence; 주제의 일관성  
: 해당 토픽모델이, 모델링이 잘 되었을수록 한 주제안에는 의미론적으로 유사한 단어가 많이 모여있게 마련이다. 따라서 상위 단어간의 유사도를 계산하면 실제로 해당 주제가 의미론적으로 일치하는 단어들끼리 모여있는지 알 수 있다. 

** 주의할 점 **
coherence가 너무 높아지면 정보의 양이 줄어들게 되고, 너무 낮아지면 인과성이 없어 분석의 의미가 낮아진다.  


---
# LDA 모델에 들어갈 객체 
- id2 word : Dictionary에 list of list of str 형식의 document를 입력하면 Dictionary가 학습된다.  
- corpus : 트윗 리스트 안의 단어를 bag-of-words 형태 -> list of(token_id, token_count) 2-tuple 로 변환  

전체 Abstract에 등장하는 단어들 통틀어서 10번 이하로 등장한 단어는 의미 없다고 판단 => 삭제 

In [142]:
id2word = corpora.Dictionary(noun_list)
id2word.filter_extremes(no_below = 10)  # 10번 이하로 등장한 단어는 삭제

texts = noun_list
corpus = [id2word.doc2bow(text) for text in texts]

In [143]:
corpus[1][1:5]   # 두번째 영상Abstract의 단어별 빈도수 예시 

[(4, 2), (9, 1), (15, 1), (36, 2)]

In [144]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=4, step=2):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word, random_state = 1234)  # 매번 토픽 바뀔 수 있으니 꼭 시드 설정해주기!! 
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts= noun_list, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values


In [145]:
model_list, coherence_values = compute_coherence_values(dictionary = id2word, corpus = corpus, texts = texts, start = 3, step=1, limit = 15)

In [146]:
limit=15; start=3; step=1;
x = range(start, limit, step)
topic_num = 0
count = 0
max_coherence = 0
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", cv)
    coherence = cv
    if coherence >= max_coherence:
        max_coherence = coherence
        topic_num = m
        model_list_num = count   
    count = count+1

Num Topics = 3  has Coherence Value of 0.26100345452988455
Num Topics = 4  has Coherence Value of 0.2600327232786418
Num Topics = 5  has Coherence Value of 0.2760552126646314
Num Topics = 6  has Coherence Value of 0.27441300350211745
Num Topics = 7  has Coherence Value of 0.28762689745835485
Num Topics = 8  has Coherence Value of 0.26798847516411417
Num Topics = 9  has Coherence Value of 0.27406583249419975
Num Topics = 10  has Coherence Value of 0.27747115173764625
Num Topics = 11  has Coherence Value of 0.2807461601733178
Num Topics = 12  has Coherence Value of 0.2799957625254829
Num Topics = 13  has Coherence Value of 0.27678952326326384
Num Topics = 14  has Coherence Value of 0.2805419067504313


In [147]:
# Select the model and print the topics
optimal_model = model_list[model_list_num]
model_topics = optimal_model.show_topics(formatted=False)
#print(optimal_model.print_topics(num_words=10))

In [150]:
def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #ldamodel[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num,topn=10)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    print(type(sent_topics_df))

    # Add original text to the end of the output
    #contents = pd.Series(texts)
    #sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df = pd.concat([sent_topics_df, df['Title'], df['Abstract']], axis=1)
    return(sent_topics_df)

In [151]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=noun_list)  	

<class 'pandas.core.frame.DataFrame'>


In [152]:
# Format
df_topic_tweet = df_topic_sents_keywords.reset_index()
df_topic_tweet.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Title','Abstract']

In [153]:
df_topic_tweet

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Title,Abstract
0,0,5.0,0.9828,"nb, soil, water, policy, space, ecosystem, inf...",Evolutions of nature based solutions for storm...,stormwater management is one of the greatest c...
1,1,2.0,0.3383,"water, ecosystem, nb, climate, site, plant, we...",Climate vulnerability and adaptation challenge...,wine producers face several challenges regardi...
2,2,3.0,0.4811,"water, ecosystem, biodiversity, infrastructure...",Technical approach and strategic plan for larg...,terrestrial ecosystems are the core components...
3,3,2.0,0.4319,"water, ecosystem, nb, climate, site, plant, we...",Mainstreaming nature-based solutions for clima...,despite a growing recognition of the importan...
4,4,5.0,0.5952,"nb, soil, water, policy, space, ecosystem, inf...",Future response of global coastal wetlands to ...,the response of coastal wetlands to sea level ...
...,...,...,...,...,...,...
1032,1032,4.0,0.8031,"ecosystem, soil, climate, water, policy, infra...",Nature-based solutions to climate change mitig...,nature based solutions promoting green and blu...
1033,1033,5.0,0.7803,"nb, soil, water, policy, space, ecosystem, inf...",Key insights for the future of urban ecosystem...,understanding the dynamics of urban ecosystem ...
1034,1034,6.0,0.9843,"ecosystem, soil, restoration, risk, carbon, ve...",A two-step strategy for developing cultivated ...,based on a site experiment on a typical steppe...
1035,1035,6.0,0.3978,"ecosystem, soil, restoration, risk, carbon, ve...",Cascades of green: A review of ecosystem-based...,climate change impacts increase pressure on ch...


In [154]:
# Show각 문서에 대한 토픽
df_topic_tweet = df_topic_tweet.sort_values(by=['Dominant_Topic'])
df_topic_tweet

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Title,Abstract
244,244,0.0,0.5578,"climate, ecosystem, water, specie, land, lands...",Impact of roof rain water harvesting of runoff...,in recent years the occurrence of floods is o...
757,757,0.0,0.9821,"climate, ecosystem, water, specie, land, lands...",A Stepwise Approach to Beach Restoration at Ca...,sea hazards are increasingly threatening world...
759,759,0.0,0.9522,"climate, ecosystem, water, specie, land, lands...",Greencities – monitoring cities for nature bas...,cities are unique living entities that evolve ...
465,465,0.0,0.6027,"climate, ecosystem, water, specie, land, lands...",Capability of shallow open-water unit for emer...,nature based solutions of polishing treated ef...
574,574,0.0,0.9512,"climate, ecosystem, water, specie, land, lands...",Coupling breakwalls with oyster restoration st...,interest and investment in constructing living...
...,...,...,...,...,...,...
918,918,6.0,0.5186,"ecosystem, soil, restoration, risk, carbon, ve...",Nature-based solutions for climate change miti...,nature conservation measures can contribute su...
920,920,6.0,0.4970,"ecosystem, soil, restoration, risk, carbon, ve...",Urbanization effect on spatiotemporal thermal ...,urban heat island uhi shows thermal intensif...
171,171,6.0,0.9872,"ecosystem, soil, restoration, risk, carbon, ve...",Vulnerability to sea-level rise and the potent...,coastal wetlands are vulnerable to sea level r...
497,497,6.0,0.6254,"ecosystem, soil, restoration, risk, carbon, ve...",International experience in territorial ecolog...,carrying out territorial ecological restoratio...


df_topic_tweet 변수 해석  
- Document_No. 문서 번호  
- Dominant_Topic : 해당되는 토픽  
- Topic_perc_Contrib : 해당 토픽의 기여도 

In [155]:
# Group top 5 sentences under each topic

sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

In [156]:
sent_topics_sorteddf_mallet  # 각 토픽별 기여 가장 많이 한 데이터 추출 

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Title,Abstract
367,0.0,0.9923,"climate, ecosystem, water, specie, land, lands...",Understanding stream bank erosion and depositi...,agricultural activities such as row cropping a...
387,1.0,0.9899,"water, risk, scale, ecosystem, flood, health, ...",Multiscale watershed landscape infrastructure:...,conventional centralized drainage systems are ...
900,2.0,0.9933,"water, ecosystem, nb, climate, site, plant, we...",Quantitative evaluation and optimized utilizat...,the water resources water environment carrying...
529,3.0,0.9932,"water, ecosystem, biodiversity, infrastructure...",Recent progress of agro-biodiversity conservat...,background agro biodiversity is one of the mo...
1014,4.0,0.9916,"ecosystem, soil, climate, water, policy, infra...",Air contaminants and litter fall decomposition...,background urban forests are usually affected ...
310,5.0,0.9890,"nb, soil, water, policy, space, ecosystem, inf...",Bayesian Belief Networks for Integrating Scien...,there is a growing interest worldwide on the p...
197,6.0,0.9925,"ecosystem, soil, restoration, risk, carbon, ve...",Does soil bioengineering benefits aquatic biod...,riverine ecosystems form a dendritic network i...


### 추가할 불용어  
[change, area, ... ] => 추가함  

In [158]:
# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)


topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
topic_counts.sort_index(inplace=True)

topic_contribution = round(topic_counts/topic_counts.sum(), 4)
topic_contribution

lda_inform = pd.concat([sent_topics_sorteddf_mallet, topic_counts, topic_contribution], axis=1)
lda_inform.columns= ["Topic_Num", "Topic_Perc_Contrib", "Keywords", 'Title','Abstract',"Num_Documents", "Perc_Documents"]
lda_inform = lda_inform[["Topic_Num","Keywords","Num_Documents","Perc_Documents"]]
lda_inform
#lda_inform.Topic_Num = lda_inform.Topic_Num.astype(int)
lda_inform['Topic_Num'] =lda_inform['Topic_Num'] +1
lda_inform.Topic_Num = lda_inform.Topic_Num.astype(str)
lda_inform['Topic_Num'] =lda_inform['Topic_Num'].str.split('.').str[0]
df_topic_tweet['Dominant_Topic'] =df_topic_tweet['Dominant_Topic'] +1
df_topic_tweet.Dominant_Topic = df_topic_tweet.Dominant_Topic.astype(str)
df_topic_tweet['Dominant_Topic'] =df_topic_tweet['Dominant_Topic'].str.split('.').str[0]

In [159]:
# 각 토픽별 문서 개수, 전체 비율 
lda_inform 

,Topic_Num,Keywords,Num_Documents,Perc_Documents
0.0,1,"climate, ecosystem, water, specie, land, lands...",117,0.1128
1.0,2,"water, risk, scale, ecosystem, flood, health, ...",157,0.1514
2.0,3,"water, ecosystem, nb, climate, site, plant, we...",158,0.1524
3.0,4,"water, ecosystem, biodiversity, infrastructure...",167,0.1610
4.0,5,"ecosystem, soil, climate, water, policy, infra...",140,0.1350
5.0,6,"nb, soil, water, policy, space, ecosystem, inf...",186,0.1794
6.0,7,"ecosystem, soil, restoration, risk, carbon, ve...",112,0.1080


In [160]:
df_topic_tweet

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Title,Abstract
244,244,1,0.5578,"climate, ecosystem, water, specie, land, lands...",Impact of roof rain water harvesting of runoff...,in recent years the occurrence of floods is o...
757,757,1,0.9821,"climate, ecosystem, water, specie, land, lands...",A Stepwise Approach to Beach Restoration at Ca...,sea hazards are increasingly threatening world...
759,759,1,0.9522,"climate, ecosystem, water, specie, land, lands...",Greencities – monitoring cities for nature bas...,cities are unique living entities that evolve ...
465,465,1,0.6027,"climate, ecosystem, water, specie, land, lands...",Capability of shallow open-water unit for emer...,nature based solutions of polishing treated ef...
574,574,1,0.9512,"climate, ecosystem, water, specie, land, lands...",Coupling breakwalls with oyster restoration st...,interest and investment in constructing living...
...,...,...,...,...,...,...
918,918,7,0.5186,"ecosystem, soil, restoration, risk, carbon, ve...",Nature-based solutions for climate change miti...,nature conservation measures can contribute su...
920,920,7,0.4970,"ecosystem, soil, restoration, risk, carbon, ve...",Urbanization effect on spatiotemporal thermal ...,urban heat island uhi shows thermal intensif...
171,171,7,0.9872,"ecosystem, soil, restoration, risk, carbon, ve...",Vulnerability to sea-level rise and the potent...,coastal wetlands are vulnerable to sea level r...
497,497,7,0.6254,"ecosystem, soil, restoration, risk, carbon, ve...",International experience in territorial ecolog...,carrying out territorial ecological restoratio...


In [161]:
optimal_model.print_topics(num_words=20)   

[(0,
  '0.019*"climate" + 0.016*"ecosystem" + 0.015*"water" + 0.012*"specie" + 0.011*"land" + 0.009*"landscape" + 0.009*"year" + 0.009*"development" + 0.008*"scenario" + 0.008*"restoration" + 0.008*"condition" + 0.008*"nb" + 0.008*"community" + 0.007*"tree" + 0.007*"plant" + 0.006*"case" + 0.006*"paper" + 0.006*"wastewater" + 0.006*"environment" + 0.006*"design"'),
 (1,
  '0.014*"water" + 0.013*"risk" + 0.012*"scale" + 0.011*"ecosystem" + 0.011*"flood" + 0.010*"health" + 0.010*"climate" + 0.010*"development" + 0.009*"strategy" + 0.009*"vegetation" + 0.009*"nb" + 0.008*"carbon" + 0.008*"design" + 0.008*"land" + 0.007*"plant" + 0.007*"policy" + 0.007*"space" + 0.006*"paper" + 0.006*"landscape" + 0.006*"challenge"'),
 (2,
  '0.032*"water" + 0.018*"ecosystem" + 0.017*"nb" + 0.010*"climate" + 0.009*"site" + 0.009*"plant" + 0.009*"wetland" + 0.008*"quality" + 0.008*"resource" + 0.008*"roof" + 0.008*"resilience" + 0.007*"specie" + 0.007*"condition" + 0.007*"space" + 0.007*"biodiversity" + 0.0

In [162]:
# txt 파일로 결과 저장
f = open('/content/drive/MyDrive/NbS/LDA_topic_20words.txt', 'w')

f.write('----- All -----')
f.write('\n')
f.write(str(optimal_model.print_topics(num_words=20)))
    
f.close()

# Step5-1. 연도 구분해서 토픽모델링  
- 토픽의 개수는 기존 LDA 에서 나온 토픽 개수대로 5개로 진행  

In [ ]:
from gensim import models
from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
import numpy
from gensim.matutils import hellinger

In [ ]:
df.head()

,Title,Year,Source title,Abstract,Author Keywords,Index Keywords,Abstract_token,Abstract_NN
0,Evolutions of nature based solutions for storm...,2020,Techniques - Sciences - Methodes,stormwater management is one of the greatest c...,NaN,cost-benefit analysis; maintenance; project as...,"[(stormwater, NN), (management, NN), (is, VBZ)...","[stormwater, challenge, environment, way, envi..."
1,Climate vulnerability and adaptation challenge...,2021,Climate,wine producers face several challenges regardi...,Adaptation; Lock-in; Viticulture; Vulnerability,NaN,"[(wine, NN), (producers, NNS), (face, VBP), (s...","[wine, producer, challenge, climate, industry,..."
2,Technical approach and strategic plan for larg...,2021,Chinese Journal of Applied Ecology,terrestrial ecosystems are the core components...,Ecocivilization; Ecological security pattern; ...,ecological approach; environmental issue; envi...,"[(terrestrial, JJ), (ecosystems, NNS), (are, V...","[ecosystem, core, component, life, production,..."
3,Mainstreaming nature-based solutions for clima...,2021,Landscape and Urban Planning,despite a growing recognition of the importan...,NaN,climate change; infrastructure planning; natur...,"[(despite, IN), (a, DT), (growing, VBG), (reco...","[recognition, importance, infrastructure, ecos..."
4,Future response of global coastal wetlands to ...,2018,Nature,the response of coastal wetlands to sea level ...,NaN,coastal waters; letter; occupation; sea level ...,"[(the, DT), (response, NN), (of, IN), (coastal...","[response, wetland, rise, twenty, century, sca..."


In [ ]:
uniqueyears, time_slices = np.unique(df.Year, return_counts=True)
print(uniqueyears, time_slices)

[2003 2006 2007 2009 2010 2012 2014 2015 2016 2017 2018 2019 2020 2021
 2022] [  1   1   1   1   1   2   5   7  20  51  73 123 263 485  22]


In [ ]:
# num_topics에 원하는 토픽 수 넣기 
# 오래 걸림 
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=id2word, time_slice= time_slices, num_topics=3, random_state = 1234)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldaseqmodel.py:290: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)


In [ ]:
from gensim.models import LdaSeqModel
from gensim.test.utils import datapath

temp_file = datapath("model")
ldaseq.save(temp_file)

ldaseq = LdaSeqModel.load(temp_file)

f = open('/content/drive/MyDrive/NbS/DTM_LDA_topic_20words.txt', 'w')

for i in range(15):
    f.write('\n')
    f.write('----- Year:  -----')
    f.write('\n')
    f.write(str(ldaseq.print_topics(time=i)))
    
f.close()

# Step6. visualization and interpretation of results

### LDAvis 사용 방법 및 해석  
1. 왼쪽 Intertopic Distance Map  
: 토픽은 단어 개수의 차원을 가지고 있음 => 2차원 시각화를 위해 차원 축소 방법 이용  
: PCA를 이용하여 n_terms 차원의 벡터를 2차원으로 압축  
: 원의 크기 => 토픽의 단어들이 얼마나 속해 있고 어떻게 분포되어 있는지를 의미한다.  
: 원의 거리 => 토픽 간의 유사성 (두 개의 원이 겹친다면, 두 개의 토픽은 유사한 토픽이라는 의미  
: 버불 중 하나로 커서를 이동하면 오른쪽의 단어와 막대가 업데이트된다.  

2. Top-30 Most Relevant Terms for Topic  
: 막대 => 토픽을 형성하는 주도적인 키워드 확인, 키워드에 커서를 올리면 해당 키워드와 관련된 토픽 확인 가능. keyword extraction(키워드 추출) 기준 2가지  
=> 1. salience  한 토픽의 키워드라면, 각 토픽에 속한 많은 문서들에서 등장해야 함  즉, P(W|t)가 커야한다. (단어w를 갖고 있는 모든 문서들 중 토픽t가 할당된 비율)  
 (문제점) p(w|t)가 가장 높은 단어는 은,는,이,가와 같은 문법적인 단어일 가능성이 높고 이는 유의한 결과가 아님. 즉, 차별성이 없는 단어는 키워드로 부적합 

 => 2. discriminative power 는 salience의 문제점을 해결하기 위해 P(w|t)를 p(w)로 나눔. 즉, 한 토픽에서 자주 등장하는 단어라 하더라고 본래 자주 등장하는 단어라면 그 중요도를 낮추겠다는 의미  
 (문제점) 최고의 discriminative power를 지닌 단어는 infrequent terms(드문 단어)일 가능성이 높음. 한 토픽에서만 등장한 단어는 전체에서도 몇 번 등장하지 않을 가능성이 높다. 즉, salience와 discriminative power 사이에는 negative correlation이 있다.  

 이 문제를 보완하기 위해 LDAvis에서는 람다값을 조절해서 두 관점의 중요도를 사람이 직접 정할 수 있다.  

---
### 람다 
: 오른쪽 상단 [0.1] 조절 가능  
: 키워드 랭킹(막대그래프 순서)은 다음의 점수로 계산한다.  
: λ⋅P(w|t)+(1−λ)⋅P(w|t) P(w)λ⋅P(w|t)+(1−λ)⋅P(w|t) P(w)  
: λ = 1일수록, 토픽 별로 가장 자주 등장하는 단어들을 우선적으로 키워드로 선택한다는 의미  
: λ = 0일수록, 토픽 간에 차이가 많이 나는 단어를 선택한다는 의미 (해당 토픽에서 많이 등장하는 단어) 


In [ ]:
!pip install gensim

In [ ]:
!pip install pyldavis

In [ ]:
from gensim.models import LdaModel
import pyLDAvis.gensim_models


def mallet_to_lda(optimal_model):
    model_gensim = gensim.models.ldamodel.LdaModel(
        id2word=optimal_model.id2word, num_topics=optimal_model.num_topics,
        alpha=optimal_model.alpha, eta=0, iterations=1000,
        gamma_threshold=0.001,
        dtype=np.float32
    )
    
    model_gensim.sync_state()
    # model_gensim.state.sstats = optimal_model.wordtopics
    return model_gensim

model = mallet_to_lda(optimal_model)

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.000814  0.000037       1        1  30.790714
0     -0.001055 -0.000804       2        1  19.369330
1      0.000018  0.001309       3        1  18.718759
3      0.000670  0.000398       4        1  17.387294
2      0.001180 -0.000940       5        1  13.733902, topic_info=                Term       Freq      Total Category  logprob  loglift
597           return  65.000000  65.000000  Default  30.0000  30.0000
863          kingdom  66.000000  66.000000  Default  29.0000  29.0000
174  denitrification  71.000000  71.000000  Default  28.0000  28.0000
252           specie  64.000000  64.000000  Default  27.0000  27.0000
783           centre  64.000000  64.000000  Default  26.0000  26.0000
..               ...        ...        ...      ...      ...      ...
44        assessment  11.404062  71.769393   Topic5  -6.6262   0.1458
129             lack  11.477782  73.111583   Topic5  -6.6198   0.1337
661             step  11.324366  71.694987   Topic5  -6.6332   0.1398
665         greening  11.493398  78.623461   Topic5  -6.6184   0.0624
282          germany  11.288445  72.090870   Topic5  -6.6364   0.1312

[250 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
831       1  0.289910  absence
831       2  0.207079  absence
831       3  0.220884  absence
831       4  0.165663  absence
831       5  0.124247  absence
...     ...       ...      ...
336       1  0.306673    world
336       2  0.213338    world
336       3  0.160003    world
336       4  0.186670    world
336       5  0.133336    world

[1105 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 2, 4, 3])

In [ ]:



coherence_values=[]

for i in range(2,15):
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=i, id2word=dictionary)   # 모델 구축 
    coherence_model_lda = CoherenceModel(model=ldamodel, texts= noun_list, dictionary=dictionary ,topn=10)
    coherence_lda = coherence_model_lda.get_coherence()
    coherence_values.append(coherence_lda)

PyKoSpacing 패키지  
- 띄어쓰기가 되어있지 않은 문장을 띄어쓰기를 한 문장으로 변환해주는 패키지  
- 설치에 문제가 있지만, 애초에 Abstract에 문장부호가 없어서 잘 되지 않는다고 한다.  

kss 한국어 문장 분리기  
- 특정 어휘(ex. 처음)에서 이상하게 분리를 하고, 모든 문장을 나눠주지는 않는다고 한다.  
- 그래도 어느정도는 작동한다고 한다.  

종결어미 확인하기 위해 mecab, kkma, okt 사용할 수 있는데 case를 나눌 것이 너무 많다.  
- kkma 는 품사 분류표가 너무 복잡, pos로 나눌 때 명사가 포함되면 종결어미인지 분간이 잘 안된다.    
- okt는 품사 분류표가 간단하지만 생각보다 case가 많다.  

# Topic Modeling (LDA)  
- n_components : 사전에 설정해주는 토픽의 개수  
- doc_topic_prior : 문서의 토픽 분포의 초기 하이퍼파라미터  
- topic_word_prior : 토픽의 단어 분포의 초기 하이퍼파라미터  
- max_iter : 반복 수행횟수  

In [ ]:
ex = df['Abstract'][0]
ex

'수식 투자 심플하게 충분하게먼저 게 너 지진희가 달라지는 계기가생겨 거요 가요제 특수 놈이 의원자력발전 루 들까 부인을 많이했었는데 2월달에 결정이 갔어요 뭐언제 표 쪼는 넣어준 걸어주는 에어우크라이나 은지 전쟁 미리 나왔구요러시아에서 찾아 가스로 계속 공급받아유럽 뿌리 이제 이게 전쟁도 있고여러가지 접속이 불안 때문에 에너지독립을 하려는 이유로 국가들을 쉬워점점 많아져요 처 구성도 좋고 수주들도 예상이 되는데 주가가 할까요[음악]안녕하세요 제식 잡학사전 여의도100 가 정소민 입니다 녁 입니다인 프로님 또 오늘은 어떤 주제가 지보셨나요 바로 알려 주시죠 북 들어오시네요 4구독 쩍 양명 서머 이제 가지고 있는원자력 발전에 대해서 이번 편에얘기해볼까 합니다 주기적으로 이원전이 이슈 거 리로 좀 떠오르고거든요 특히 연초에 이 원전 관련 이주들이 한테서 트 석회 있어요 왜그랬는지 배경 설명 좀 해주시죠 그원적 에너지 지위가 달라지는 계획을생각해요 가요 이제 그 유행 4기협약 이라는게 일단 있구요 뒤에서벽초 외로워 c 5 p 라고 해 왠지1990년에 어떤 기본 협약 을체결하고 들어가서 1차 회의 전체총애를 이제 95년에 있었구요copd 에 이제 번호가 붙어요 edcope 배차 아 1 2 제 3의 뭐3회차 이렇게 되는거 에서 제가cop 삶이며 는 도 또 다음에 빠리 기 우 협약이 쉬 op.22 디어써요 이고 이제 일본의 cop 26이 작년에 열렸어요 이제 석탄 의단계적 감축 을 얘기 한데요 2월에얘기했던 게 그 이후 특수 놈이라고있어요 낸 것들이 이제 어떤 책에분류체계를 얘기하는 건데그동안은 여기 그 특선 어미의 원자력발전으로 들까봐 lng 어쩌는가 건고민을 많이 했었는데 2월달에 결정이갔어요 언제 회전을 넣어 준 걸넣어주는 에어 넣어주는 거래도 발표가되어서 이게 그린 색소 놈이라고 이분류기준 확정이 됐구요 4 다만 이번지역발전을 2004 15년까지 만행규 건설하게 해주고 호 이후에는 이제완전하게 100% 신재생에너지로 만가겠다 아 이런 가도 비전 이유로원자력발전소를

0     [음악]
1      갔어요
2      같아요
3      거든요
4      거예요
      ... 
59      하죠
60     할까요
61     합니다
62     해야죠
63    했었는데
Name: 불용어, Length: 64, dtype: object

In [ ]:

tokenizer=RegexpTokenizer("[\w]+")

In [ ]:
count={}
tokens=tokenizer.tokenize(ex)
len(tokens)

2345

In [ ]:
tokens

In [ ]:
stopped_tokens=[i for i in list(set(tokens)) if not i in stop_words+["br"]]
stopped_tokens

In [ ]:
stopped_tokens2=[i for i in stopped_tokens if len(i)>1]
for i, a in enumerate(stopped_tokens2):
    for b in stopped_tokens2[i+1:]:
        if a>b:
            count[b, a]=count.get((b, a),0)+1
        else :
            count[a, b]=count.get((a, b),0)+1
df=pd.DataFrame.from_dict(count, orient='index')

In [ ]:
df

,0
"(2017년까지, 탁수)",1
"(오늘은, 탁수)",1
"(안정성이, 탁수)",1
"(전격, 탁수)",1
"(들까봐, 탁수)",1
...,...
"(에너지가사업을, 추진로켓을)",1
"(발전을, 에너지가사업을)",1
"(이잖아요, 추진로켓을)",1
"(발전을, 이잖아요)",1


In [ ]:
list1=[]
for i in range(len(df)):
    list1.append([df.index[i][0],df.index[i][1],df[0][i]])

In [ ]:
df2=pd.DataFrame(list1,columns=["term1","term2","freq"])
df3=df2.sort_values(by=["freq"],ascending=False)
df3_pos=df3.reset_index(drop=True)

,term1,term2,freq
0,2017년까지,탁수,1
1,독일과,특히,1
2,독일과,악보도하고,1
3,독일과,째로사용하긴,1
4,관문전이나,독일과,1


In [ ]:
df3_pos

,term1,term2,freq
0,2017년까지,탁수,1
1,독일과,특히,1
2,독일과,악보도하고,1
3,독일과,째로사용하긴,1
4,관문전이나,독일과,1
...,...,...,...
1194280,문제로,입에,1
1194281,올라,입에,1
1194282,입에,하면은핵분열이,1
1194283,그럼,입에,1


In [ ]:
import networkx as nx
import operator

import numpy as np
G_pos=nx.Graph()

for i in range((len(np.where(df3_pos['freq']>10)[0]))):
    G_pos.add_edge(df3_pos['term1'][i], df3_pos['term2'][i], weight=int(df3_pos['freq'][i]))

dgr = nx.degree_centrality(G_pos)
btw = nx.betweenness_centrality(G_pos)
cls = nx.closeness_centrality(G_pos)
egv = nx.eigenvector_centrality(G_pos)

sorted_dgr = sorted(dgr.items(), key=operator.itemgetter(1), reverse=True)
sorted_btw = sorted(btw.items(), key=operator.itemgetter(1), reverse=True)
sorted_cls = sorted(cls.items(), key=operator.itemgetter(1), reverse=True)
sorted_egv = sorted(egv.items(), key=operator.itemgetter(1), reverse=True)

print("** degree **")
for x in range(10):
    print(sorted_dgr[x])

print("** betweenness **")
for x in range(10):
    print(sorted_btw[x])

print("** closeness **")
for x in range(10):
    print(sorted_cls[x])

print("** eigenvector **")
for x in range(10):
    print(sorted_egv[x])

NetworkXPointlessConcept: ignored